# DSSD数据分析

## 目的：
 
- 根据相邻条关联和正背面关联，确认多重触发事件的入射位置、能量，并重新组织事件
- 根据位置-能量关联，对事件进行分类
- 得到多重粒子入射事件中，每个事件的pid，能量，位置等信息。

## 数据：

- d1-d3探测器的幅度已归一化。
- hit结构中能量按照由大到小排序。

## 观察事件多重性

```c++
tree->Draw("y1hit:x1hit>>(6,0,6,6,0,6)","","colz");
gPad->SetLogz();
c1->Draw();
```

## 用Scan观察事件特征，找到分析思路

**事件的整体特征**

- 事件2，一个粒子入射
  - x:相邻两重，y:相邻两重；x总能量(3485)，y总能量很接近(3480)
- 事件3，4，5，6，8，10，一个粒子入射
  - x，y:一重和相邻两重的组合；x能量，y能量很接近
- 事件1，7
  - x，y:一重和相邻两重的组合；x能量与y能量相差40左右
- 事件9，
  - x:三重，其中有两条相邻(13,14)，y:相邻两重(12,13)；x,y面相邻条总能量相近，x面另一条(10)?
  - 两侧总能量不一致。

**非相邻两重事件**

- 显示的非相邻两重事件，的x-y都有很好的关联,可以明确指定位置组合。

**相邻两重事件**

- 很多相邻两重事件都是由两个入射粒子引起的。

**三重事件**

也可以清晰地看见有三个粒子入射的情况:

- 事件118:粒子1(18,3),粒子2(12,28),粒子3(10,11)
- 事件193：
其他：

- 事件192:粒子1(2,14),粒子2((12,11),(6,5,7))
- 事件398:x和y方向都有相邻条，但每条之间x，y能量都有很好的关联，因此很大概率是三个入射粒子。
- 事件9,289等两侧总能量不一致。

**总结**

1. 正背面关联关系可以对事件类型(是否为多重，相邻条crosstalk)做很好的区分。
2. 当两侧总能量不一致时，可以认为事件为而非正常事件。
  - 部分能量没有被记录，或有偶然符合
  - 非正常事件，做标记后待后续处理确认。
3.新参数：
  - 每个探测器x方向总能量：sx1e,sx2e,sx3e;
  - y方向总能量：sy1e,sy2e,sy3e;

## 画图观察，并确定x-y关联的能量差范围

**总结** 
- 当两侧多重性不一致时：sx1e-sy1e 中心值不为零，向正(x1hit>y1hit),负(x1hit<y1hit)的一侧偏移
  - 意味着在能量沉积大时，有些条的感应信号因为阈值(硬件pedal的阈值)没有被记录
  - 但这个问题实际并不严重：在能量接近8000的时候最大偏移在50左右，对能量的相对影响为 ΔE/E=50/8000≈0.006 , 小于1%。
- 对于多粒子入射引起的x,y多重性事件：abs(sx1e-sy1e)<10
  - 当相邻条满足上述条件，也认为是两个粒子入射导致的。

## 形成新文件
- 重新对事件进行分类，确定每个事件的实际多重性(多重性=入射粒子数目)

**示例代码**

```c++
Int_t x1h,y1h,x2h,y2h,x3h,y3h;//final multiplicity
  Double_t x1ea[100],x2ea[100],x3ea[100];//x-energy
  Double_t y1ea[100],y2ea[100],y3ea[100];//y-energy
  Int_t x1sa[100],x2sa[100],x3sa[100];//x-strip
  Int_t y1sa[100],y2sa[100],y3sa[100];//y-strip
  Int_t flag1,flag2,flag3;//event flag
////////////////////////////////////
      //one-one DSSD1
   if(x1hit==1 && y1hit==1 && abs(sx1e-sy1e)<10) {
        x1h=1;y1h=1;
        x1ea[0]=x1e[0]; x1sa[0]=x1s[0];
        y1ea[0]=y1e[0]; y1sa[0]=y1s[0];
        flag1=111;
     }
////////////////////////////////////

////////////////////////////////////
      //two-two DSSD1
   if(x1hit==y1hit && x1hit==2) {
        if(abs(x1e[0]-y1e[0])<10 && abs(x1e[1]-y1e[1])<10) {
            x1h=2;y1h=2;
            for(int i=0;i<2;i++) {
                x1ea[i]=x1e[i]; x1sa[i]=x1s[i];
                y1ea[i]=y1e[i]; y1sa[i]=y1s[i];
            }
        flag1=222;
        }
        else if(sx1e-sy1e>-10 && sx1e-sy1e<60) {
            if(abs(x1s[0]-x1s[1])==1 && abs(y1s[0]-y1s[1])==1) {
                x1h=1;y1h=1;
                x1ea[0]=x1e[0]+x1e[1];
                x1sa[0]=x1s[0];
                y1ea[0]=y1e[0]+y1e[1];
                y1sa[0]=y1s[0];
                flag1=221;
              }
            }      
    }
//////////////////////////////////////
//three-two DSSD1
  if(x1hit==2 && y1hit==3) {
    if(sx1e-sy1e>-35 && sx1e-sy1e<10) {
      int ix=-1,iy=-1;
      for(int i=0;i<2;i++) 
        for(int j=0;j<3;j++) {
          if(abs(x1e[i]-y1e[j])<10) {//search x-y pair
            ix=i;
            iy=j;
           }
         }
      if(ix>-1) {
        x1sa[0]=-1; x1sa[1]=-1;
        y1sa[0]=-1; y1sa[1]=-1;

        x1ea[0]=x1e[ix]; x1sa[0]=x1s[ix];
        y1ea[0]=y1e[iy]; y1sa[0]=y1s[iy];

        if(ix==0) {
          x1ea[1]=x1e[1]; x1sa[1]=x1s[1];
        }
        else {
          x1ea[1]=x1e[0]; x1sa[1]=x1s[0];
        }
        if(iy==0 &&abs(y1s[1]-y1s[2])==1) {
          y1ea[1]=y1e[1]+y1e[2]; y1sa[1]=y1s[1];
        }
        if(iy==1 &&abs(y1s[0]-y1s[2])==1) {
          y1ea[1]=y1e[0]+y1e[2]; y1sa[1]=y1s[0];
        }
        if(iy==2 &&abs(y1s[0]-y1s[1])==1) {
          y1ea[1]=y1e[0]+y1e[1]; y1sa[1]=y1s[0];
        }
        if(x1sa[0]>-1 && x1sa[1]>-1 &&y1sa[0]>-1 && y1sa[1]>-1) {
          x1h=2;y1h=2;
          flag1=232;
        }
      }
    }
  }

 if(x1hit==3 && y1hit==2) {
    if(sx1e-sy1e>5 && sx1e-sy1e<65) {
      int ix=-1,iy=-1;
      for(int i=0;i<3;i++) 
        for(int j=0;j<2;j++) {
          if(abs(x1e[i]-y1e[j])<10) {//search x-y pair
            ix=i;
            iy=j;
          }
        }
      if(ix>-1) {
        x1sa[0]=-1; x1sa[1]=-1;
        y1sa[0]=-1; y1sa[1]=-1;
        x1ea[0]=x1e[ix]; x1sa[0]=x1s[ix];
        y1ea[0]=y1e[iy]; y1sa[0]=y1s[iy];

        if(iy==0) {
          y1ea[1]=y1e[1]; y1sa[1]=y1s[1];
        }
        else {
          y1ea[1]=y1e[0]; y1sa[1]=y1s[0];
        }
        if(ix==0 &&abs(x1s[1]-x1s[2])==1) {
          x1ea[1]=x1e[1]+x1e[2]; x1sa[1]=x1s[1];
        }
        if(ix==1 &&abs(x1s[0]-x1s[2])==1) {
          x1ea[1]=x1e[0]+x1e[2]; x1sa[1]=x1s[0];
        }
        if(ix==2 &&abs(x1s[0]-x1s[1])==1) {
          x1ea[1]=x1e[0]+x1e[1]; x1sa[1]=x1s[0];
        }
        if(x1sa[0]>-1 && x1sa[1]>-1 &&y1sa[0]>-1 && y1sa[1]>-1) {
          x1h=2;y1h=2;
          flag1=232;
        }
      }
    }
  }

... ... ...
 if((x1h>0 && x1h==x2h && x3h==0) || (x1h>0 && x1h==x2h && x2h==x3h)) tree->Fill();
```

## 检查事件特征，如PID

## 遗留问题

### 按照单个探测器的x-y关联不能完全确定不同探测器之间的事件之间对应关系
- 事件 60，958
  - 60： x1s-x2s错位
  - 958： x1s-x2s错位
- 探测器间事件匹配错位，导致de-e图上的正确关联关系外的散点。

### 需要进行trak matching

按照如下条件重新进行多探测器之间事件匹配
- 探测器之间的位置不能相差太多，否则为粒子探测器上的散射或反应。
- 多个探测器位置构建的位置要指向靶的有效区域。

### 部分事件无法进行trak matching
- 事件277：两个事件的x1s和x2s几乎一致，无法进行track matching
### 最终还要通过pid matching确定匹配顺序
- 观察哪种排列方法符合正确的pid关系


# 小结

- DSSD要多信息、多关联、多观察；
- 结合自己的物理目标，灵活寻找、调整重构策略；
- 数据处理永无止境，结合物理目标、个人能力寻找终点；

In [2]:
!jupyter nbconvert example3_6.ipynb --to html

[NbConvertApp] Converting notebook example3_6.ipynb to html

[NbConvertApp] Writing 319365 bytes to example3_6.html

